In [1]:
import torch
import torch.nn as nn
from Model import model, modelArguments
import time
import re
import math
import triton


In [2]:
fmodel = model(modelArguments)

state_dict = torch.load('./checkpoints/model_weightsBPE.pth')
new_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
fmodel.load_state_dict(new_state_dict, strict=False)

fmodel = fmodel.to('cuda')

/tmp/ipykernel_519698/2760450671.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./checkpoints/model_weightsBPE.pth')


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="large_tokenizer.json")


def tokenize(text, second = False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids1 = ids
    ids1 = torch.tensor(ids1, dtype=torch.int16)
    
    # # Pad or truncate the sequence to tholtae max_seq_length
    # if ids1.size(0) < modelArguments.max_seq_len:
    #     padding_length = modelArguments.max_seq_len - ids1.size(0)
    #     ids1 = torch.cat([ids1, torch.zeros(padding_length, dtype=torch.int16)])
        
    # elif ids1.size(0) >= modelArguments.max_seq_len:
    #     ids1 = ids1[:modelArguments.max_seq_len]
    #     ids1[-1] = 2
    return ids1

In [4]:
def id_to_token(x):
    s = ''
    for i in x[0]:
        # j = torch.argmax(i)
        s += tokenizer.decode(i)

    return s

def id_to_token_M(x):
    s = ''
    for i in x[0]:
        j = torch.argmax(i)
        s += tokenizer.decode(j)

    return s

In [5]:
fmodel.eval()

model(
  (shared_emb): shared_emb()
  (enc_layers): ModuleList(
    (0-9): 10 x EncoderBlock(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attention): FlashAttention(
        (q_linear): Linear(in_features=512, out_features=512, bias=False)
        (k_linear): Linear(in_features=512, out_features=512, bias=False)
        (v_linear): Linear(in_features=512, out_features=512, bias=False)
        (out_linear): Linear(in_features=512, out_features=512, bias=False)
      )
      (feedforward): FeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=False)
        (fc2): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [102]:
# test sentence

sentences = ["""Summarize the given paragraph \n Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the forefront of technological advancements. \n\n""",
            
            """Given the paragraph where is Ayush Sharma born \n Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the forefront \n\n""",
             
             """Given the paragraph what are Ayush Sharma's skills \n Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the \n\n"""
            ]


generated_ids = []
for i in sentences:
    generated_id  = torch.tensor(tokenizer.encode(i)).to(torch.long).to('cuda').unsqueeze(0)
    generated_ids.append(generated_id)



generated_ids = torch.cat(generated_ids)


# generated_ids = torch.tensor(generated_ids)

In [103]:
generated_ids.shape

torch.Size([3, 110])

In [104]:


print(generated_ids.shape)
# print(generated_ids)
# generated_ids = generated_id.repeat(4, 1)

t0 = time.time()

for _ in range(modelArguments.max_seq_len-generated_ids.size(1)):  # Generate up to 20 tokens
    with torch.no_grad():
        logits = fmodel(generated_ids)  # Pass the current sequence to the model
        next_token_logits = logits[:, -1, :]  # Get logits for the last token


    # Predict the next token (greedy decoding)
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(1)  # Shape: [1, 1]

    # Append the predicted token to the sequence
    generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

    # Stop if EOS token is predicted
    # if next_token_id.item() == 2:
    #     print("eos")
    #     break

t1 = time.time()
# Decode the generated sequence
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)
t = (t1-t0)*1000
print("time taken ", t)
# generated_ids = generated_id.repeat(2, 1)


torch.Size([3, 110])
Generated text: Summarize the given paragraph 
 Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the forefront of technological advancements. 

The Indian-language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language lang

In [105]:
generated_ids.shape

torch.Size([3, 256])

In [106]:
next_token_id.shape

torch.Size([3, 1])

In [107]:
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)

Generated text: Summarize the given paragraph 
 Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the forefront of technological advancements. 

The Indian-language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language language languag

In [108]:
output_text = tokenizer.decode(generated_ids[1])
print("Generated text:", output_text)

Generated text: Given the paragraph where is Ayush Sharma born 
 Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the forefront 

The Ayush Sharma is a dynamic and dynamic company that has gained significant attention in the field of technology, technology, and technology His approach to innovation has led to a significant shift in the field of technology, and he has been a part of the Indian community
The Ayush Sharma is a renowned leader in the field of technology, and he has been instrumental in shaping the future of the field of technology His dedication to innovation and innovation has led to a significant tr

In [109]:
output_text = tokenizer.decode(generated_ids[2])
print("Generated text:", output_text)

Generated text: Given the paragraph what are Ayush Sharma's skills 
 Ayush Sharma, also known as Frost Head, is a dynamic and forward-thinking individual with a deep passion for technology. With a multifaceted expertise that spans deep learning, AI, and web development, Ayush combines technical prowess, creative thinking, and problem-solving finesse. Born in Mandi, Himachal Pradesh, India, Ayush has been immersed in the world of programming since a young age, honing his skills and staying at the 

The Indian Eye of Ayush Sharma is a powerful and powerful tool that can be used to enhance the overall experience of the Indian Eye of Medicine, and it is a powerful tool that can be used to enhance the overall experience of the Eye of Medicine
The Indian Eye of Ayush Sharma is a powerful tool that has gained popularity in the Indian eye, and it has been used to enhance the overall experience of Ayush Sharma, a powerful tool that can be used to enhance the overall experience of the Indian Eye

In [110]:
# output_text = tokenizer.decode(generated_ids[3])
# print("Generated text:", output_text)